In [ ]:
print("Start")

In [ ]:
import importlib

def detect_google_adk():
    try:
        adk = importlib.import_module("google_adk")
        print("✅ Detected package: google_adk")
        return "google_adk"
    except ModuleNotFoundError:
        pass

    try:
        adk = importlib.import_module("google.adk")
        print(" Detected package: google.adk")
        return "google.adk"
    except ModuleNotFoundError:
        pass

    print("Neither 'google_adk' nor 'google.adk' found. Check your installation.")
    return None

pkg = detect_google_adk()


In [ ]:
try:
    from google_adk.agents import LlmAgent
    from google_adk.models.lite_llm import LiteLlm
    print("✅ Imported from google_adk")
except ModuleNotFoundError:
    try:
        from google.adk.agents import LlmAgent
        from google.adk.models.lite_llm import LiteLlm
        print("✅ Imported from google.adk")
    except ModuleNotFoundError as e:
        print("❌ Import failed:", e)


In [ ]:
from google.adk.agents import LlmAgent
from google.adk.models.lite_llm import LiteLlm
from dotenv import load_dotenv
from litellm import completion
import json


In [ ]:
import os
from dotenv import load_dotenv


env_path = r"C:\Users\cyril\OneDrive\Desktop\ITC\Project3\SAGE\.env"

# Load environment variables from that path
load_dotenv(dotenv_path=env_path)

print("✅ Loaded .env from:", env_path)
print("✅ OPENAI_API_KEY loaded?", bool(os.getenv("OPENAI_API_KEY")))


In [ ]:
def analyze_sentiment(conversation: list) -> dict:
    import re, json
    from litellm import completion

    def safe_parse_json(raw):
        cleaned = re.sub(r"^```(?:json)?|```$", "", raw.strip(), flags=re.MULTILINE).strip()
        try:
            return json.loads(cleaned)
        except json.JSONDecodeError:
            return None

    timeline = []

    for entry in conversation:
        start_t, end_t, speaker, text = entry

        resp = completion(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a precise sentiment analysis model. "
                        "Return a JSON object: {\"label\": <emotion>, \"score\": <0–1>} "
                        "Respond ONLY in JSON without markdown or code blocks."
                    ),
                },
                {"role": "user", "content": text},
            ],
        )

        raw = resp["choices"][0]["message"]["content"]
        parsed = safe_parse_json(raw)

        if parsed:
            label = parsed.get("label", "neutral")
            score = float(parsed.get("score", 0.5))
        else:
            label, score = "neutral", 0.5

        timeline.append({"t": start_t, "label": label, "score": score})

    labels = [t["label"] for t in timeline]
    overall = max(set(labels), key=labels.count)

    return {
        "sentiment_overall": overall,
        "granularity": "1m",
        "start_time": conversation[0][0],
        "timeline": timeline,
    }


In [ ]:
sentiment_agent = LlmAgent(
    model=LiteLlm(model="openai/gpt-4o"),
    name="sentiment_agent",
    description="Analyzes emotional tone across conversation turns.",
    instruction=(
        "Perform fine-grained sentiment analysis on text segments. "
        "Return structured JSON with overall sentiment and a timeline."
    ),
    tools=[analyze_sentiment],
)


In [ ]:
from dotenv import load_dotenv
load_dotenv()


In [ ]:
# ------------------ SAMPLE RUN ------------------
if __name__ == "__main__":
    conversation = conversation = [
    [
        "2025-10-24T09:00:00Z",
        "2025-10-24T09:00:59Z",
        "Customer",
        "Good morning. I just got a late payment fine of ₹1,200 on my credit card, but I’m sure I paid the bill yesterday. Can you please tell me why this fine was charged?"
    ],
    [
        "2025-10-24T09:01:00Z",
        "2025-10-24T09:01:59Z",
        "Customer Care",
        "Good morning! I’m sorry for the trouble. Let me check your account details. Could you please confirm the payment date and amount so I can verify the transaction?"
    ],
    [
        "2025-10-24T09:02:00Z",
        "2025-10-24T09:02:59Z",
        "Customer",
        "Yes, I paid ₹18,500 yesterday around 10 a.m. through the mobile app. It showed successful, so I don’t understand why this fine was added today."
    ]
]


    result = analyze_sentiment(conversation)
    print(json.dumps(result, indent=2))

In [ ]:
print("done")